In [1]:
import numpy as np 
import scipy
from scipy.integrate import nquad

'''Change the below values for your coil'''
#'''
coil_turns = 20
coil_length = 0.5 #units in cm, side-on measured
coil_diameter = 0.1 #units in cm from top down
coil_radius =coil_diameter/2
wire_thickness = 0.05 #units of cm,diameter
wire_radius = wire_thickness/2
#'''

#Inductance of coil the below are defaults
'''
coil_turns = 3
coil_length = 0.7 #units in cm, side-on measured
coil_diameter = 0.5 #units in cm from top down
coil_radius =coil_diameter/2
wire_thickness = 0.69 #units of cm,diameter
wire_radius = wire_thickness/2
'''

'''Other constants and values we need'''
medium_permeability =  np.pi*4*10**-7 #vacuum value
coil_area = (np.pi)*(coil_diameter**2)/4
coax_imp = 50 #Coax cable impedance in Ohms
Q = 210 #quality factor of coil, Q


#max and min capacitances of your variable capacitors in pF
C_match_1 = np.array([2, 120]) 
C_match_2 = np.array([1, 1]) #1pf fixed capacitor
C_tune = np.array([2, 120])

#24/10/2024 Adding In coaxial cable additional capacitance
#Length of coaxial cable
L=0.90 #Length in m, including bending section length, between connector at probe top and cable termination at probe base
C_per_meter=90 #capacitance per meter in pF (100pF/m is rough, find the exact size for your cable in documentation)
#Adding in cable capacitance for the coax down the probe centre:
cable_capacitance = L*C_per_meter*10**(-12)

#convert to F
C_match_tot = C_match_1*10**-12 + C_match_2*10**-12 + cable_capacitance
C_tune = C_tune*10**-12


In [5]:
'''Inductance of coil calculation'''
'''NB. This cell uses cubepy which did not allow the computation by default, its run time must be manually changed in the package code to be 64 not 16'''

#14/11/2024: if you want to use the simple equation this is also calculated. It is only set to be used if 'SIMPLE=True' and 'COMPLEX=False' the inverse will print only the complex one and use that.
Simple = True
Advanced = False

'''Advanced calculation'''
import cubepy as cp
## The integrand function.
def function_to_integrate(phi, p_phi, theta, p_theta):
    #R=1

    a=wire_radius
    b=coil_radius
    h=coil_length
    N=coil_turns

    k= h/(2*N*np.pi)
    permeability = np.pi*4*10**-7
    coefficient = permeability/(4*np.pi*(2*np.pi*a)**2)
    
    r1 = (b*np.sin(theta) + a*np.cos(phi)*np.sin(theta) - b*np.sin(p_phi) - a*np.cos(p_phi)*np.sin(p_theta))
    r2=(a*np.sin(phi) + k*theta - a*np.sin(p_phi) - k*p_theta)
    r3 = (b*np.cos(theta) +a*np.cos(phi)*np.cos(theta) - b*np.cos(p_theta) - a*np.cos(p_phi)*np.cos(p_theta))
    R = np.sqrt(r1**2 + r2**2 + r3**2)
    #R=b*np.sin(theta)+ a*np.cos(phi)
    f1 = (np.cos(p_theta - theta)*a**2)/R
    f2 = b+a*np.cos(phi)
    f3 = b+a*np.cos(p_phi)

    integrand = coefficient*f1*f2*f3
    return integrand
N=coil_turns


'''Simple calculation (see 'inductance_calculator.ipynb' for more details, from digikey website)'''
a=wire_radius
b=coil_radius
h=coil_length
N=coil_turns
permeability = np.pi*4*10**-7

R=h/(a*permeability)
simple_value=(N**2)/R


#14/11/2024 adding in step for choosing which equation to use:
if Simple:
    print(f'''simple value: {simple_value}H, error: N/A H, 
    coil_turns = {coil_turns}, 
    coil_length = {coil_length} #units in cm, side on measured, 
    coil_diameter = {coil_diameter} #units in cm from top down, 
    coil_radius = {coil_diameter/2}, 
    wire_thickness = {wire_thickness} #units of cm,diameter, 
    wire_radius = {wire_thickness/2}''')

if Advanced:
    ## The upper and lower bounds of integration.
    low = [0.0, 0.0, 0.0, 0.0]
    high = [2*np.pi, 2*np.pi, N*2*np.pi, N*2*np.pi]
    # The Integration!
    complex_value, error = cp.integrate(function_to_integrate, low, high)
    
    print(f'''complex value: {complex_value}H, error: {error}H, 
    coil_turns = {coil_turns}, 
    coil_length = {coil_length} #units in cm, side on measured, 
    coil_diameter = {coil_diameter} #units in cm from top down, 
    coil_radius = {coil_diameter/2}, 
    wire_thickness = {wire_thickness} #units of cm,diameter, 
    wire_radius = {wire_thickness/2}''')

if Advanced and not Simple:
    ## The upper and lower bounds of integration.
    low = [0.0, 0.0, 0.0, 0.0]
    high = [2*np.pi, 2*np.pi, N*2*np.pi, N*2*np.pi]
    # The Integration!
    complex_value, error = cp.integrate(function_to_integrate, low, high)
    
    coil_L = complex_value

if Simple and not Advanced:
    coil_L = simple_value

if Simple and Advanced:
    print(f'Cannot assign a value of inductance, choose if you want to use the simple or advanced value at the top of this cell')



simple value: 2.5132741228718347e-05H, error: N/A H, 
    coil_turns = 20, 
    coil_length = 0.5 #units in cm, side on measured, 
    coil_diameter = 0.1 #units in cm from top down, 
    coil_radius = 0.05, 
    wire_thickness = 0.05 #units of cm,diameter, 
    wire_radius = 0.025


Code to show range of possible resonance frequencies for given conditions in NMR circuit for series parallel and parallel series setups.



In [7]:
#pick your circuit type by making it True and the other False
Series_parallel = False
Parallel_series = True



if Series_parallel and not Parallel_series:
    res_f = (1/(2*np.pi) * 1/np.sqrt(coil_L*(C_match_tot+C_tune)))*10**-6 #resonant freq in MHz
    print(f'Series Parallel resonant frequency range is: {res_f}MHz')

if not Series_parallel and Parallel_series:
    C_tot = 1/(C_match_tot**-1 +C_tune**-1)
    res_f = (1/(2*np.pi) * 1/np.sqrt(coil_L*C_tot))*10**-6 #resonant freq in MHz
    print(f'Parallel Series resonant frequency range is: {res_f}MHz')

elif Series_parallel and Parallel_series:
    print('Check the true false statements, one must be true, one must be false to show which circuit you are using')
elif not Series_parallel and not Parallel_series:
    print('Check the true false statements, one must be true, one must be false to show which circuit you are using')

Parallel Series resonant frequency range is: [22.71406094  3.65899512]MHz


Impedance matching section, once you know the range of your resonance condition you need to calculate the impedance matching conditions

NB. THIS ONLY WORKS FOR PARALLEL SERIES CIRCUITS CURRENTLY, CHECK NUTS AND BOLTS NMR FOR SERIES PARALLEL CONDITION

In [6]:
impedance_to_match = 50 #ohms, the impedance you want the circuit to be matched, to prevent reflection this is the source impedance (usually 50 ohms from the preamp)
internal_r_coil = 0.2 #ohms, the resistance of the Cobalt test sample wire
#internal_r_coil = 70 #ohms, the resistance of the Cl test sample wire

#Nuclei Angular frequencies for NMR (2π*γ) 
Co_0T=2*np.pi*213.16*10**6
Cl_9T=2*np.pi*37.58*10**6
Fl_per_T=2*np.pi*40.05*10**6

ang_frequency = Fl_per_T #the angular frequency you want resonance at

inductance = coil_L #inductance of the coil calculated before, in henrys 

#this code produces the matching condition in the form of the ratio of C/(C+C'), with Q=ωL/r where r is the coil resistance but when tuning using this remember that C'>>C is the ideal condition
Q = ang_frequency*inductance/internal_r_coil
print(f'Q={Q}')
capacitance_ratio = np.sqrt((impedance_to_match*internal_r_coil)/((ang_frequency**2)*(inductance**2)))


#can this be achieved with the capacitor values given earlier?
C_match_min = np.min(C_match_tot)
C_match_max = np.max(C_match_tot)
print(C_match_min, C_match_max)
C_tune_min = np.min(C_tune)
C_tune_max = np.max(C_tune)


cap_ratio_min = C_tune_min/(C_tune_min+C_match_max)
cap_ratio_max = C_tune_max/(C_tune_max+C_match_min)

if capacitance_ratio<cap_ratio_min:
    #print('true')
    text = 'therefore this matching cannot be completed, use a bigger matching capacitor or a smaller coil inductance'
if capacitance_ratio>cap_ratio_max:
    text = 'something has gone really wrong, this should never happen!'
elif capacitance_ratio<cap_ratio_max and capacitance_ratio>cap_ratio_min:
    text = 'this ratio can be achieved within the current capacitor setup'


print(f'''Capacitance ratio required: C/(C+C\') ={capacitance_ratio}, max and min possible ratios with these capacitors:{cap_ratio_max},{cap_ratio_min}, 
    {text}''')

Q=31622.2125010903
8.400000000000001e-11 2.02e-10
Capacitance ratio required: C/(C+C') =0.000500008919372632, max and min possible ratios with these capacitors:0.588235294117647,0.00980392156862745, 
    therefore this matching cannot be completed, use a bigger matching capacitor or a smaller coil inductance
